In [1]:
import numpy as np
import pandas as pd
import pymongo
import base64
import string

In [15]:
import random

In [2]:
df = pd.read_csv('base_municipios_IDH_PIB_LOC')

In [3]:
df.columns

Index(['Unnamed: 0', 'Nome da Unidade da Federação', 'IDHM', 'IDHM Educação',
       'IDHM Longevidade', 'IDHM Renda', 'Sigla da Unidade da Federação',
       'Nome do Município', 'PIB', 'PIB Per Capita', 'população estimada',
       'codigo_ibge', 'latitude', 'longitude'],
      dtype='object')

In [4]:
df = df.drop(columns=['Unnamed: 0'])

In [5]:
df.columns

Index(['Nome da Unidade da Federação', 'IDHM', 'IDHM Educação',
       'IDHM Longevidade', 'IDHM Renda', 'Sigla da Unidade da Federação',
       'Nome do Município', 'PIB', 'PIB Per Capita', 'população estimada',
       'codigo_ibge', 'latitude', 'longitude'],
      dtype='object')

In [6]:
df.head()

,Nome da Unidade da Federação,IDHM,IDHM Educação,IDHM Longevidade,IDHM Renda,Sigla da Unidade da Federação,Nome do Município,PIB,PIB Per Capita,população estimada,codigo_ibge,latitude,longitude
0,Rondônia,0.641,0.526,0.763,0.657,RO,Alta Floresta D'Oeste,262076.878,10731.18,24422,1100015,-11.92830,-61.9953
1,Rondônia,0.702,0.600,0.806,0.716,RO,Ariquemes,1364693.780,15103.86,90353,1100023,-9.90571,-63.0325
2,Rondônia,0.650,0.559,0.757,0.650,RO,Cabixi,69611.114,11033.62,6309,1100031,-13.49450,-60.5520
3,Rondônia,0.718,0.620,0.821,0.727,RO,Cacoal,1186494.091,15095.15,78601,1100049,-11.43430,-61.4562
4,Rondônia,0.692,0.602,0.799,0.688,RO,Cerejeiras,222021.189,13037.06,17030,1100056,-13.18700,-60.8168


In [7]:
columns = ['stateName', 'idhm', 'idhmEducation', 'idhmLongevity', 'idhmIncome', 'state', 'city', 'gdp', 'gdpPerCapita', 'population', 'ibge', 'latitude', 'logintude']

In [8]:
df.columns = columns

In [9]:
client = pymongo.MongoClient(base64.b64decode('').decode(), ssl=True)  

In [10]:
def id_generator(size=10, chars=string.ascii_uppercase + string.digits + string.ascii_lowercase):
    return ''.join(random.choice(chars) for _ in range(size))

In [11]:
df.head()

,stateName,idhm,idhmEducation,idhmLongevity,idhmIncome,state,city,gdp,gdpPerCapita,population,ibge,latitude,logintude
0,Rondônia,0.641,0.526,0.763,0.657,RO,Alta Floresta D'Oeste,262076.878,10731.18,24422,1100015,-11.92830,-61.9953
1,Rondônia,0.702,0.600,0.806,0.716,RO,Ariquemes,1364693.780,15103.86,90353,1100023,-9.90571,-63.0325
2,Rondônia,0.650,0.559,0.757,0.650,RO,Cabixi,69611.114,11033.62,6309,1100031,-13.49450,-60.5520
3,Rondônia,0.718,0.620,0.821,0.727,RO,Cacoal,1186494.091,15095.15,78601,1100049,-11.43430,-61.4562
4,Rondônia,0.692,0.602,0.799,0.688,RO,Cerejeiras,222021.189,13037.06,17030,1100056,-13.18700,-60.8168


In [13]:
from tqdm import tqdm

In [17]:
df.shape

(5564, 13)

In [ ]:
for document in tqdm(df.to_dict(orient='records')):
    
    # Mudando IDs
    counted = 100
    while counted != 0:
        _id = id_generator()
        counted = client['gyramais']['socioEconomic'].count_documents({ '_id' : _id})

    if client['gyramais']['socioEconomic'].count_documents({ 'ibge' : document['ibge']}) > 0:

        _id = list(client['gyramais']['socioEconomic'].find({ 'ibge' : document['ibge']}, { '_id' : True}))[0]['_id']
    
    document['_id'] = _id
    
    #document['_acl'] = {}
    #document['_acl']['superuser'] = {'w': True, 'r': True} 
    #document['_acl']['administrator'] = {'r': True}
    #document['_rperm'] = ['superuser', 'administrator'],
    #document['_wperm'] = ['superuser']
    
    client['gyramais']['socioEconomic'].replace_one({'_id':document['_id']}, document, True)
    
client.close()

  7%|▋         | 364/5564 [04:26<1:00:06,  1.44it/s]